# 58. 特徴点検出とマッチング
## Feature Detection and Matching

---

## 学習目標

このノートブックを完了すると、以下ができるようになります：

- [ ] 特徴点検出の基本原理（コーナー検出）を理解する
- [ ] Harris コーナー検出を実装できる
- [ ] スケール不変性の概念と SIFT の原理を理解する
- [ ] 特徴量記述子の役割と種類を説明できる
- [ ] 特徴点マッチングアルゴリズムを実装できる
- [ ] RANSAC による外れ値除去を適用できる

---

## 前提知識

- 画像処理の基礎（畳み込み、勾配）
- 55: エピポーラ幾何の理論（マッチング検証用）
- 線形代数：固有値、SVD

**難易度**: ★★★☆☆（中級）  
**推定学習時間**: 90-120分

---

## 1. 特徴点とは

**特徴点（Feature Point / Keypoint）**は、画像内で他の領域と区別可能な特徴的な点です。

### 良い特徴点の条件

| 性質 | 説明 |
|------|------|
| **繰り返し検出可能** | 異なる画像でも同じ物理的な点を検出 |
| **識別可能** | 周囲の情報で一意に特定できる |
| **局所性** | 小さな領域の情報で検出・記述 |
| **不変性** | 回転、スケール、照明変化に対してロバスト |

### 特徴点の種類

- **コーナー**: 2方向に強いエッジ（Harris, Shi-Tomasi）
- **ブロブ**: 周囲と輝度が異なる領域（SIFT, SURF）
- **エッジ**: 1方向に強い勾配（特徴点としては弱い）

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage
from scipy.ndimage import gaussian_filter
from typing import Tuple, List, Optional
import warnings
warnings.filterwarnings('ignore')

np.set_printoptions(precision=4, suppress=True)

print("ライブラリのインポート完了")

In [ ]:
def create_test_image(size: int = 256) -> np.ndarray:
    """テスト用の合成画像を生成"""
    image = np.ones((size, size)) * 128
    
    # 長方形（エッジとコーナー）
    image[50:150, 50:150] = 200
    
    # 小さな正方形
    image[180:210, 180:210] = 50
    
    # 円（ブロブ）
    y, x = np.ogrid[:size, :size]
    mask = (x - 200)**2 + (y - 80)**2 < 25**2
    image[mask] = 220
    
    # テクスチャ領域
    np.random.seed(42)
    texture = np.random.randn(60, 60) * 30
    image[60:120, 170:230] += texture
    
    # ガウシアンブラーで滑らかに
    image = gaussian_filter(image, sigma=1)
    
    return image.astype(np.float32)

# テスト画像の生成と表示
test_image = create_test_image()

plt.figure(figsize=(8, 8))
plt.imshow(test_image, cmap='gray')
plt.title('Test Image for Feature Detection')
plt.colorbar()
plt.show()

---

## 2. Harris コーナー検出

### 2.1 基本アイデア

画像パッチを少しずらしたときの変化を調べます：

- **平坦領域**: どの方向にずらしても変化なし
- **エッジ**: エッジに沿った方向では変化なし、垂直方向では変化大
- **コーナー**: どの方向にずらしても変化大

### 2.2 数学的定式化

変位 $(u, v)$ に対する二乗誤差和（SSD）：

$$E(u, v) = \sum_{x, y} w(x, y) [I(x+u, y+v) - I(x, y)]^2$$

テイラー展開で近似：

$$E(u, v) \approx \begin{pmatrix} u & v \end{pmatrix} \mathbf{M} \begin{pmatrix} u \\ v \end{pmatrix}$$

### 2.3 構造テンソル（Second Moment Matrix）

$$\mathbf{M} = \sum_{x, y} w(x, y) \begin{pmatrix} I_x^2 & I_x I_y \\ I_x I_y & I_y^2 \end{pmatrix} = \begin{pmatrix} A & B \\ B & C \end{pmatrix}$$

### 2.4 Harris レスポンス関数

$$R = \det(\mathbf{M}) - k \cdot \text{trace}(\mathbf{M})^2 = \lambda_1 \lambda_2 - k(\lambda_1 + \lambda_2)^2$$

- $R > 0$（大）: コーナー
- $R < 0$（大）: エッジ
- $|R|$ 小: 平坦領域

In [ ]:
def compute_image_gradients(image: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    """画像の勾配を計算（Sobel演算子）"""
    # Sobelカーネル
    sobel_x = np.array([[-1, 0, 1],
                        [-2, 0, 2],
                        [-1, 0, 1]]) / 8.0
    
    sobel_y = np.array([[-1, -2, -1],
                        [0, 0, 0],
                        [1, 2, 1]]) / 8.0
    
    Ix = ndimage.convolve(image.astype(float), sobel_x)
    Iy = ndimage.convolve(image.astype(float), sobel_y)
    
    return Ix, Iy

def harris_corner_detection(image: np.ndarray, 
                             k: float = 0.04,
                             window_size: int = 5,
                             threshold: float = 0.01) -> Tuple[np.ndarray, List[Tuple[int, int]]]:
    """Harris コーナー検出
    
    Args:
        image: 入力画像（グレースケール）
        k: Harris レスポンスのパラメータ（通常 0.04-0.06）
        window_size: ガウシアン窓のサイズ
        threshold: コーナー判定の閾値（最大レスポンスに対する割合）
    
    Returns:
        R: Harris レスポンスマップ
        corners: コーナー座標のリスト [(y, x), ...]
    """
    # 1. 勾配の計算
    Ix, Iy = compute_image_gradients(image)
    
    # 2. 構造テンソルの要素
    Ixx = Ix ** 2
    Iyy = Iy ** 2
    Ixy = Ix * Iy
    
    # 3. ガウシアン重みで平滑化
    sigma = window_size / 6.0
    A = gaussian_filter(Ixx, sigma=sigma)
    B = gaussian_filter(Ixy, sigma=sigma)
    C = gaussian_filter(Iyy, sigma=sigma)
    
    # 4. Harris レスポンスの計算
    det_M = A * C - B ** 2
    trace_M = A + C
    R = det_M - k * (trace_M ** 2)
    
    # 5. Non-Maximum Suppression
    R_max = ndimage.maximum_filter(R, size=window_size)
    R_nms = (R == R_max) & (R > threshold * R.max())
    
    # 6. コーナー座標の抽出
    corners = list(zip(*np.where(R_nms)))
    
    return R, corners

print("Harris コーナー検出の実装完了")

In [ ]:
# Harris コーナー検出の実行
harris_response, corners = harris_corner_detection(test_image, k=0.04, threshold=0.01)

print(f"検出されたコーナー数: {len(corners)}")

# 結果の可視化
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# 元画像
axes[0].imshow(test_image, cmap='gray')
axes[0].set_title('Original Image')
axes[0].axis('off')

# Harris レスポンス
im = axes[1].imshow(harris_response, cmap='jet')
axes[1].set_title('Harris Response')
axes[1].axis('off')
plt.colorbar(im, ax=axes[1], fraction=0.046)

# コーナー検出結果
axes[2].imshow(test_image, cmap='gray')
for y, x in corners:
    axes[2].plot(x, y, 'r.', markersize=8)
axes[2].set_title(f'Detected Corners ({len(corners)} points)')
axes[2].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
def visualize_harris_eigenvalues():
    """Harris 検出の固有値空間での解釈"""
    fig, ax = plt.subplots(figsize=(8, 8))
    
    # 固有値空間
    lambda1 = np.linspace(0, 1, 100)
    lambda2 = np.linspace(0, 1, 100)
    L1, L2 = np.meshgrid(lambda1, lambda2)
    
    # Harris レスポンス
    k = 0.04
    R = L1 * L2 - k * (L1 + L2) ** 2
    
    # プロット
    contour = ax.contourf(L1, L2, R, levels=50, cmap='RdBu_r')
    ax.contour(L1, L2, R, levels=[0], colors='black', linewidths=2)
    
    plt.colorbar(contour, label='Harris Response R')
    
    # 領域のラベル
    ax.text(0.1, 0.1, 'Flat\n(R ≈ 0)', fontsize=12, ha='center', va='center',
            bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    ax.text(0.8, 0.1, 'Edge\n(R < 0)', fontsize=12, ha='center', va='center',
            bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    ax.text(0.1, 0.8, 'Edge\n(R < 0)', fontsize=12, ha='center', va='center',
            bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    ax.text(0.8, 0.8, 'Corner\n(R > 0)', fontsize=12, ha='center', va='center',
            bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.8))
    
    ax.set_xlabel('λ₁ (First Eigenvalue)', fontsize=12)
    ax.set_ylabel('λ₂ (Second Eigenvalue)', fontsize=12)
    ax.set_title('Harris Response in Eigenvalue Space', fontsize=14)
    ax.set_aspect('equal')
    
    plt.tight_layout()
    plt.show()

visualize_harris_eigenvalues()

---

## 3. スケール不変特徴点（SIFT の概要）

### 3.1 スケール不変性の必要性

Harris コーナー検出は**スケール変化に弱い**です。同じオブジェクトでも、画像のスケールが変わると検出位置が変わってしまいます。

### 3.2 SIFT (Scale-Invariant Feature Transform)

David Lowe が2004年に提案した、スケールと回転に不変な特徴点検出・記述手法。

#### ステップ1: スケール空間の構築

$$L(x, y, \sigma) = G(x, y, \sigma) * I(x, y)$$

異なるスケール $\sigma$ でガウシアンブラーを適用。

#### ステップ2: DoG (Difference of Gaussian)

$$D(x, y, \sigma) = L(x, y, k\sigma) - L(x, y, \sigma)$$

DoG はラプラシアン・オブ・ガウシアン（LoG）の近似で、ブロブ検出に有効。

#### ステップ3: 極値検出

DoG 空間で $(x, y, \sigma)$ の26近傍（同スケール8点 + 上下スケール各9点）で極値を検出。

#### ステップ4: キーポイントの精緻化

サブピクセル精度で位置を補正し、低コントラスト点やエッジ上の点を除去。

#### ステップ5: 方向の割り当て

キーポイント周辺の勾配ヒストグラムから主方向を決定。

In [ ]:
def build_gaussian_pyramid(image: np.ndarray, 
                            n_octaves: int = 4, 
                            n_scales: int = 5,
                            sigma: float = 1.6) -> List[List[np.ndarray]]:
    """ガウシアンピラミッドの構築"""
    k = 2 ** (1.0 / (n_scales - 3))  # スケール間の比率
    
    pyramid = []
    current_image = image.copy()
    
    for octave in range(n_octaves):
        octave_images = []
        current_sigma = sigma
        
        for scale in range(n_scales):
            if scale == 0 and octave > 0:
                # オクターブ間でダウンサンプリング
                blurred = gaussian_filter(current_image, sigma=current_sigma)
            else:
                blurred = gaussian_filter(current_image, sigma=current_sigma)
            
            octave_images.append(blurred)
            current_sigma *= k
        
        pyramid.append(octave_images)
        
        # 次のオクターブ用にダウンサンプリング
        current_image = octave_images[-3][::2, ::2]  # 2倍ダウンサンプリング
    
    return pyramid

def build_dog_pyramid(gaussian_pyramid: List[List[np.ndarray]]) -> List[List[np.ndarray]]:
    """Difference of Gaussian (DoG) ピラミッドの構築"""
    dog_pyramid = []
    
    for octave_images in gaussian_pyramid:
        dog_octave = []
        for i in range(len(octave_images) - 1):
            dog = octave_images[i + 1] - octave_images[i]
            dog_octave.append(dog)
        dog_pyramid.append(dog_octave)
    
    return dog_pyramid

# ピラミッドの構築と可視化
gaussian_pyr = build_gaussian_pyramid(test_image, n_octaves=3, n_scales=5)
dog_pyr = build_dog_pyramid(gaussian_pyr)

# 最初のオクターブを可視化
fig, axes = plt.subplots(2, 5, figsize=(15, 6))

for i, img in enumerate(gaussian_pyr[0]):
    axes[0, i].imshow(img, cmap='gray')
    axes[0, i].set_title(f'Gaussian σ={i}')
    axes[0, i].axis('off')

for i, img in enumerate(dog_pyr[0]):
    axes[1, i].imshow(img, cmap='RdBu_r')
    axes[1, i].set_title(f'DoG {i}')
    axes[1, i].axis('off')

axes[1, 4].axis('off')

plt.suptitle('Gaussian Pyramid and Difference of Gaussian (First Octave)', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
def detect_dog_extrema(dog_pyramid: List[List[np.ndarray]], 
                        threshold: float = 0.03) -> List[Tuple[int, int, int, int]]:
    """DoG 空間での極値検出（簡略版）
    
    Returns:
        keypoints: [(octave, scale, y, x), ...]
    """
    keypoints = []
    
    for octave_idx, dog_octave in enumerate(dog_pyramid):
        for scale_idx in range(1, len(dog_octave) - 1):
            current = dog_octave[scale_idx]
            prev = dog_octave[scale_idx - 1]
            next_scale = dog_octave[scale_idx + 1]
            
            h, w = current.shape
            
            for y in range(1, h - 1):
                for x in range(1, w - 1):
                    val = current[y, x]
                    
                    if abs(val) < threshold:
                        continue
                    
                    # 26近傍チェック
                    neighbors = []
                    for dy in range(-1, 2):
                        for dx in range(-1, 2):
                            neighbors.append(prev[y + dy, x + dx])
                            if dy != 0 or dx != 0:
                                neighbors.append(current[y + dy, x + dx])
                            neighbors.append(next_scale[y + dy, x + dx])
                    
                    # 極値判定
                    if val > 0 and val >= max(neighbors):
                        keypoints.append((octave_idx, scale_idx, y, x))
                    elif val < 0 and val <= min(neighbors):
                        keypoints.append((octave_idx, scale_idx, y, x))
    
    return keypoints

# 極値検出
sift_keypoints = detect_dog_extrema(dog_pyr, threshold=0.02)
print(f"検出されたキーポイント数: {len(sift_keypoints)}")

# 可視化（最初のオクターブのみ）
fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(test_image, cmap='gray')

for octave, scale, y, x in sift_keypoints:
    if octave == 0:
        # スケールに応じた円のサイズ
        radius = 3 * (scale + 1)
        circle = plt.Circle((x, y), radius, fill=False, color='red', linewidth=1)
        ax.add_patch(circle)

ax.set_title(f'SIFT-like Keypoints (Octave 0, {sum(1 for k in sift_keypoints if k[0]==0)} points)')
ax.axis('off')
plt.tight_layout()
plt.show()

---

## 4. 特徴量記述子

### 4.1 記述子の役割

**特徴量記述子（Descriptor）**は、キーポイント周辺の情報をベクトル化したもの。マッチングに使用します。

### 4.2 SIFT 記述子

1. キーポイント周辺を 16×16 ピクセルの領域に分割
2. 各 4×4 セルで勾配方向ヒストグラム（8方向）を計算
3. 4×4×8 = 128 次元ベクトル
4. 正規化して照明変化に対するロバスト性を確保

### 4.3 その他の記述子

| 記述子 | 次元 | 特徴 |
|--------|------|------|
| **SIFT** | 128 | 高精度、計算コスト高 |
| **SURF** | 64/128 | SIFT より高速 |
| **ORB** | 256 (binary) | 非常に高速、バイナリ |
| **BRIEF** | 256 (binary) | シンプル、高速 |

In [ ]:
def compute_gradient_histogram(image: np.ndarray, 
                                y: int, x: int,
                                window_size: int = 16,
                                n_bins: int = 8) -> np.ndarray:
    """勾配方向ヒストグラムを計算（簡略版SIFT記述子）"""
    half = window_size // 2
    
    # 境界チェック
    h, w = image.shape
    if y - half < 0 or y + half >= h or x - half < 0 or x + half >= w:
        return None
    
    # パッチの抽出
    patch = image[y - half:y + half, x - half:x + half]
    
    # 勾配の計算
    Ix, Iy = compute_image_gradients(patch)
    
    # 勾配の大きさと方向
    magnitude = np.sqrt(Ix**2 + Iy**2)
    orientation = np.arctan2(Iy, Ix)  # -π to π
    
    # 4x4 セルに分割
    cell_size = window_size // 4
    descriptor = []
    
    for cy in range(4):
        for cx in range(4):
            cell_mag = magnitude[cy*cell_size:(cy+1)*cell_size,
                                  cx*cell_size:(cx+1)*cell_size]
            cell_ori = orientation[cy*cell_size:(cy+1)*cell_size,
                                    cx*cell_size:(cx+1)*cell_size]
            
            # ヒストグラム
            hist, _ = np.histogram(cell_ori.flatten(), 
                                    bins=n_bins, 
                                    range=(-np.pi, np.pi),
                                    weights=cell_mag.flatten())
            descriptor.extend(hist)
    
    descriptor = np.array(descriptor)
    
    # 正規化
    norm = np.linalg.norm(descriptor)
    if norm > 1e-6:
        descriptor = descriptor / norm
    
    # クリッピングと再正規化（SIFT の手法）
    descriptor = np.clip(descriptor, 0, 0.2)
    norm = np.linalg.norm(descriptor)
    if norm > 1e-6:
        descriptor = descriptor / norm
    
    return descriptor

def compute_descriptors(image: np.ndarray, 
                        keypoints: List[Tuple]) -> Tuple[List, np.ndarray]:
    """全キーポイントの記述子を計算"""
    descriptors = []
    valid_keypoints = []
    
    for kp in keypoints:
        if len(kp) == 4:  # SIFT-like (octave, scale, y, x)
            octave, scale, y, x = kp
            # オクターブに応じてスケールアップ
            y_scaled = y * (2 ** octave)
            x_scaled = x * (2 ** octave)
        else:  # Harris (y, x)
            y_scaled, x_scaled = kp
        
        desc = compute_gradient_histogram(image, int(y_scaled), int(x_scaled))
        
        if desc is not None:
            descriptors.append(desc)
            valid_keypoints.append((y_scaled, x_scaled))
    
    return valid_keypoints, np.array(descriptors)

# 記述子の計算
kps, descs = compute_descriptors(test_image, corners)
print(f"記述子を計算したキーポイント: {len(kps)}")
print(f"記述子の次元: {descs.shape[1] if len(descs) > 0 else 0}")

In [ ]:
def visualize_descriptor(image: np.ndarray, y: int, x: int):
    """記述子の可視化"""
    desc = compute_gradient_histogram(image, y, x)
    
    if desc is None:
        print("キーポイントが画像境界に近すぎます")
        return
    
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    # 画像とキーポイント
    axes[0].imshow(image, cmap='gray')
    axes[0].plot(x, y, 'r+', markersize=20, markeredgewidth=3)
    rect = plt.Rectangle((x-8, y-8), 16, 16, fill=False, color='red', linewidth=2)
    axes[0].add_patch(rect)
    axes[0].set_title(f'Keypoint at ({x}, {y})')
    axes[0].axis('off')
    
    # 16x16 パッチ
    patch = image[y-8:y+8, x-8:x+8]
    axes[1].imshow(patch, cmap='gray')
    # 4x4 グリッド
    for i in range(1, 4):
        axes[1].axhline(y=i*4-0.5, color='red', linewidth=1)
        axes[1].axvline(x=i*4-0.5, color='red', linewidth=1)
    axes[1].set_title('16×16 Patch with 4×4 Grid')
    axes[1].axis('off')
    
    # 記述子
    desc_reshaped = desc.reshape(4, 4, 8)
    for cy in range(4):
        for cx in range(4):
            hist = desc_reshaped[cy, cx]
            # 極座標で表示
            angles = np.linspace(-np.pi, np.pi, 9)[:-1]
            for i, (angle, mag) in enumerate(zip(angles, hist)):
                if mag > 0.01:
                    center_x = cx + 0.5
                    center_y = cy + 0.5
                    dx = mag * np.cos(angle) * 0.4
                    dy = mag * np.sin(angle) * 0.4
                    axes[2].arrow(center_x, center_y, dx, dy, 
                                  head_width=0.05, head_length=0.02, 
                                  fc='blue', ec='blue')
    
    axes[2].set_xlim(0, 4)
    axes[2].set_ylim(4, 0)
    axes[2].set_aspect('equal')
    axes[2].grid(True)
    axes[2].set_title('Descriptor (Gradient Histograms)')
    
    plt.tight_layout()
    plt.show()

# コーナーの1つで記述子を可視化
if len(corners) > 0:
    # 画像中央に近いコーナーを選択
    center_y, center_x = test_image.shape[0] // 2, test_image.shape[1] // 2
    valid_corners = [(y, x) for y, x in corners 
                     if 20 < y < test_image.shape[0]-20 and 20 < x < test_image.shape[1]-20]
    if valid_corners:
        y, x = valid_corners[0]
        visualize_descriptor(test_image, y, x)

---

## 5. 特徴点マッチング

### 5.1 最近傍マッチング

記述子間の距離（ユークリッド距離 or ハミング距離）が最小のペアをマッチとします。

### 5.2 Lowe's Ratio Test

誤マッチを減らすため、最近傍と2番目に近い点の距離比を使用：

$$\frac{d_1}{d_2} < \text{ratio\_threshold}$$

通常、ratio_threshold = 0.7-0.8 を使用。

In [ ]:
def match_descriptors(desc1: np.ndarray, desc2: np.ndarray,
                       ratio_threshold: float = 0.75) -> List[Tuple[int, int, float]]:
    """記述子のマッチング（Lowe's ratio test）
    
    Args:
        desc1: 画像1の記述子 (N1, D)
        desc2: 画像2の記述子 (N2, D)
        ratio_threshold: Lowe's ratio test の閾値
    
    Returns:
        matches: [(idx1, idx2, distance), ...]
    """
    matches = []
    
    for i, d1 in enumerate(desc1):
        # 全記述子との距離を計算
        distances = np.linalg.norm(desc2 - d1, axis=1)
        
        # 最近傍と2番目に近いものを取得
        sorted_indices = np.argsort(distances)
        
        if len(sorted_indices) < 2:
            continue
        
        best_idx = sorted_indices[0]
        second_idx = sorted_indices[1]
        
        best_dist = distances[best_idx]
        second_dist = distances[second_idx]
        
        # Lowe's ratio test
        if second_dist > 1e-6 and best_dist / second_dist < ratio_threshold:
            matches.append((i, best_idx, best_dist))
    
    return matches

def cross_check_matches(matches_12: List[Tuple], 
                        matches_21: List[Tuple]) -> List[Tuple]:
    """双方向マッチングによる検証"""
    # 逆方向マッチの辞書を作成
    reverse_matches = {m[0]: m[1] for m in matches_21}
    
    verified_matches = []
    for idx1, idx2, dist in matches_12:
        if idx2 in reverse_matches and reverse_matches[idx2] == idx1:
            verified_matches.append((idx1, idx2, dist))
    
    return verified_matches

print("マッチング関数の実装完了")

In [ ]:
def create_transformed_image(image: np.ndarray, 
                              rotation: float = 15,
                              scale: float = 0.9,
                              translation: Tuple[int, int] = (20, 10)) -> np.ndarray:
    """変換を加えた画像を生成（マッチングテスト用）"""
    from scipy.ndimage import rotate, zoom, shift
    
    # 回転
    transformed = rotate(image, rotation, reshape=False, mode='constant', cval=128)
    
    # スケーリング（パディング付き）
    if scale != 1.0:
        h, w = transformed.shape
        zoomed = zoom(transformed, scale)
        
        # 元のサイズに合わせてクロップまたはパディング
        zh, zw = zoomed.shape
        result = np.ones((h, w)) * 128
        
        if scale < 1.0:
            # パディング
            start_y = (h - zh) // 2
            start_x = (w - zw) // 2
            result[start_y:start_y+zh, start_x:start_x+zw] = zoomed
        else:
            # クロップ
            start_y = (zh - h) // 2
            start_x = (zw - w) // 2
            result = zoomed[start_y:start_y+h, start_x:start_x+w]
        
        transformed = result
    
    # 平行移動
    transformed = shift(transformed, translation, mode='constant', cval=128)
    
    return transformed.astype(np.float32)

# 変換画像の生成
image1 = test_image
image2 = create_transformed_image(test_image, rotation=10, scale=0.95, translation=(15, -10))

# 両画像で特徴点検出
_, corners1 = harris_corner_detection(image1, k=0.04, threshold=0.01)
_, corners2 = harris_corner_detection(image2, k=0.04, threshold=0.01)

# 記述子の計算
kps1, descs1 = compute_descriptors(image1, corners1)
kps2, descs2 = compute_descriptors(image2, corners2)

print(f"画像1: {len(kps1)} キーポイント")
print(f"画像2: {len(kps2)} キーポイント")

# マッチング
if len(descs1) > 0 and len(descs2) > 0:
    matches_12 = match_descriptors(descs1, descs2, ratio_threshold=0.8)
    matches_21 = match_descriptors(descs2, descs1, ratio_threshold=0.8)
    
    # 双方向検証
    verified_matches = cross_check_matches(matches_12, matches_21)
    
    print(f"\nマッチング結果:")
    print(f"  1→2 マッチ: {len(matches_12)}")
    print(f"  双方向検証後: {len(verified_matches)}")

In [ ]:
def visualize_matches(image1: np.ndarray, image2: np.ndarray,
                       kps1: List, kps2: List,
                       matches: List[Tuple],
                       max_matches: int = 50):
    """マッチングの可視化"""
    # 画像を横に並べる
    h1, w1 = image1.shape
    h2, w2 = image2.shape
    
    canvas = np.zeros((max(h1, h2), w1 + w2))
    canvas[:h1, :w1] = image1
    canvas[:h2, w1:w1+w2] = image2
    
    fig, ax = plt.subplots(figsize=(16, 8))
    ax.imshow(canvas, cmap='gray')
    
    # マッチを描画
    colors = plt.cm.hsv(np.linspace(0, 1, min(len(matches), max_matches)))
    
    for i, (idx1, idx2, dist) in enumerate(matches[:max_matches]):
        y1, x1 = kps1[idx1]
        y2, x2 = kps2[idx2]
        
        x2_shifted = x2 + w1
        
        ax.plot([x1, x2_shifted], [y1, y2], '-', color=colors[i], linewidth=1, alpha=0.7)
        ax.plot(x1, y1, 'o', color=colors[i], markersize=5)
        ax.plot(x2_shifted, y2, 'o', color=colors[i], markersize=5)
    
    ax.set_title(f'Feature Matches ({len(matches)} total, showing {min(len(matches), max_matches)})', fontsize=14)
    ax.axis('off')
    
    plt.tight_layout()
    plt.show()

if len(verified_matches) > 0:
    visualize_matches(image1, image2, kps1, kps2, verified_matches)

---

## 6. RANSAC による外れ値除去

### 6.1 問題

マッチング結果には誤対応（外れ値/outliers）が含まれます。これを除去する必要があります。

### 6.2 RANSAC アルゴリズム

```
for i = 1 to max_iterations:
    1. ランダムに最小サンプル（ホモグラフィなら4点）を選択
    2. そのサンプルからモデルを推定
    3. 全点に対してモデルを評価、インライア数をカウント
    4. インライア数が最大なら、このモデルを保持

最終的に、全インライアでモデルを再推定
```

In [ ]:
def compute_homography(src_pts: np.ndarray, dst_pts: np.ndarray) -> np.ndarray:
    """4点以上からホモグラフィを計算（DLT法）"""
    n = len(src_pts)
    A = np.zeros((2 * n, 9))
    
    for i in range(n):
        x, y = src_pts[i]
        u, v = dst_pts[i]
        
        A[2*i] = [-x, -y, -1, 0, 0, 0, u*x, u*y, u]
        A[2*i + 1] = [0, 0, 0, -x, -y, -1, v*x, v*y, v]
    
    _, _, Vt = np.linalg.svd(A)
    H = Vt[-1].reshape(3, 3)
    H = H / H[2, 2]
    
    return H

def apply_homography(H: np.ndarray, pts: np.ndarray) -> np.ndarray:
    """ホモグラフィを点に適用"""
    n = len(pts)
    pts_h = np.hstack([pts, np.ones((n, 1))])
    transformed = (H @ pts_h.T).T
    return transformed[:, :2] / transformed[:, 2:3]

def ransac_homography(src_pts: np.ndarray, dst_pts: np.ndarray,
                       threshold: float = 3.0,
                       max_iterations: int = 1000) -> Tuple[np.ndarray, np.ndarray]:
    """RANSACによるホモグラフィ推定
    
    Returns:
        H: 最良のホモグラフィ
        inlier_mask: インライアのマスク
    """
    n_points = len(src_pts)
    best_H = None
    best_inliers = np.zeros(n_points, dtype=bool)
    best_n_inliers = 0
    
    for _ in range(max_iterations):
        # 1. ランダムに4点選択
        indices = np.random.choice(n_points, 4, replace=False)
        
        # 2. ホモグラフィを計算
        try:
            H = compute_homography(src_pts[indices], dst_pts[indices])
        except:
            continue
        
        # 3. 全点に対して評価
        projected = apply_homography(H, src_pts)
        errors = np.linalg.norm(projected - dst_pts, axis=1)
        
        # 4. インライアの判定
        inliers = errors < threshold
        n_inliers = np.sum(inliers)
        
        # 5. 最良モデルの更新
        if n_inliers > best_n_inliers:
            best_n_inliers = n_inliers
            best_H = H
            best_inliers = inliers
    
    # 6. 全インライアでホモグラフィを再計算
    if np.sum(best_inliers) >= 4:
        best_H = compute_homography(src_pts[best_inliers], dst_pts[best_inliers])
    
    return best_H, best_inliers

print("RANSAC の実装完了")

In [ ]:
# マッチング点を抽出
if len(verified_matches) >= 4:
    src_pts = np.array([[kps1[m[0]][1], kps1[m[0]][0]] for m in verified_matches])  # (x, y)
    dst_pts = np.array([[kps2[m[1]][1], kps2[m[1]][0]] for m in verified_matches])
    
    # RANSACでホモグラフィを推定
    H, inlier_mask = ransac_homography(src_pts, dst_pts, threshold=5.0, max_iterations=1000)
    
    print(f"マッチ数: {len(verified_matches)}")
    print(f"インライア数: {np.sum(inlier_mask)}")
    print(f"外れ値数: {np.sum(~inlier_mask)}")
    print(f"\n推定されたホモグラフィ:")
    print(H)
    
    # インライアのみのマッチを可視化
    inlier_matches = [m for m, is_inlier in zip(verified_matches, inlier_mask) if is_inlier]
    
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # 全マッチ
    h1, w1 = image1.shape
    h2, w2 = image2.shape
    canvas1 = np.zeros((max(h1, h2), w1 + w2))
    canvas1[:h1, :w1] = image1
    canvas1[:h2, w1:] = image2
    
    axes[0].imshow(canvas1, cmap='gray')
    for idx1, idx2, _ in verified_matches:
        y1, x1 = kps1[idx1]
        y2, x2 = kps2[idx2]
        axes[0].plot([x1, x2 + w1], [y1, y2], 'r-', alpha=0.5, linewidth=1)
    axes[0].set_title(f'All Matches ({len(verified_matches)})')
    axes[0].axis('off')
    
    # インライアのみ
    canvas2 = canvas1.copy()
    axes[1].imshow(canvas2, cmap='gray')
    for idx1, idx2, _ in inlier_matches:
        y1, x1 = kps1[idx1]
        y2, x2 = kps2[idx2]
        axes[1].plot([x1, x2 + w1], [y1, y2], 'g-', alpha=0.7, linewidth=1)
    axes[1].set_title(f'Inliers Only ({len(inlier_matches)})')
    axes[1].axis('off')
    
    plt.suptitle('RANSAC: Outlier Removal', fontsize=14)
    plt.tight_layout()
    plt.show()
else:
    print("マッチが不足しています")

---

## 7. 各手法の比較

### 7.1 特徴点検出器の比較

| 手法 | スケール不変 | 回転不変 | 速度 | 用途 |
|------|------------|---------|------|------|
| **Harris** | × | △ | 高速 | リアルタイム |
| **SIFT** | ○ | ○ | 遅い | 高精度マッチング |
| **SURF** | ○ | ○ | 中程度 | バランス |
| **ORB** | △ | ○ | 非常に高速 | モバイル/リアルタイム |
| **FAST** | × | × | 最速 | SLAM |

### 7.2 実用上の選択指針

- **精度重視**: SIFT > SURF > ORB
- **速度重視**: FAST > ORB > SURF > SIFT
- **メモリ重視**: ORB（バイナリ記述子）

---

## 8. まとめと次のステップ

### 学んだこと

1. **特徴点の概念**: 繰り返し検出可能で識別可能な点
2. **Harris コーナー検出**: 構造テンソルの固有値解析
3. **スケール不変性**: SIFT のガウシアンピラミッドとDoG
4. **特徴量記述子**: 勾配ヒストグラムによる128次元ベクトル
5. **マッチング**: 最近傍探索とLowe's ratio test
6. **RANSAC**: 外れ値除去によるロバスト推定

### 重要な数式

| 概念 | 数式 |
|------|------|
| 構造テンソル | $\mathbf{M} = \sum w \begin{pmatrix} I_x^2 & I_x I_y \\ I_x I_y & I_y^2 \end{pmatrix}$ |
| Harris レスポンス | $R = \det(\mathbf{M}) - k \cdot \text{trace}(\mathbf{M})^2$ |
| DoG | $D = L(x, y, k\sigma) - L(x, y, \sigma)$ |
| Ratio test | $d_1 / d_2 < \text{threshold}$ |

### 次のノートブック

**59. SfM パイプライン基礎**では：
- Structure from Motion の全体像
- インクリメンタル SfM
- 初期化とカメラ追加

---

## 9. 自己評価クイズ

以下の質問に答えて理解度を確認しましょう：

1. Harris コーナー検出で、構造テンソル $\mathbf{M}$ の2つの固有値がともに大きい場合、その点は何を意味しますか？

2. SIFT がスケール不変性を持つ仕組みを説明してください。

3. Lowe's ratio test とは何ですか？なぜ有効なのですか？

4. RANSAC がホモグラフィ推定に必要な理由は？

5. ORB が SIFT より高速な理由は何ですか？

6. 特徴量記述子の「正規化」が重要な理由は？

In [ ]:
# クイズの解答（隠し）
def show_quiz_answers():
    answers = """
    === 自己評価クイズ解答 ===
    
    1. 両固有値が大きい場合:
       - 2つの主方向ともに強い変化がある
       - これはコーナー（2方向にエッジがある点）を意味する
       - Harris レスポンス R が大きな正の値になる
    
    2. SIFT のスケール不変性:
       - ガウシアンピラミッドで複数スケールの画像を生成
       - DoG（Difference of Gaussian）でブロブを検出
       - スケール空間で極値を探すことで、特徴的なスケールを決定
       - 検出されたスケールに応じて記述子を計算
    
    3. Lowe's ratio test:
       - 最近傍と2番目に近い点の距離比を計算
       - 比が閾値未満の場合のみマッチとして採用
       - 有効な理由: 正しいマッチは2番目より明らかに近い
       - 曖昧なマッチ（複数の似た候補がある）を排除
    
    4. RANSAC が必要な理由:
       - 特徴点マッチングには誤対応（外れ値）が含まれる
       - 外れ値があると最小二乗法が大きく歪む
       - RANSAC はロバストに正しいモデルを推定
       - インライアとアウトライアを分離できる
    
    5. ORB が高速な理由:
       - バイナリ記述子（256ビット）を使用
       - ハミング距離で高速に比較可能（XORとポップカウント）
       - FAST コーナー検出器ベース（非常に高速）
       - 浮動小数点演算が少ない
    
    6. 記述子の正規化が重要な理由:
       - 照明変化への対応（全体的な明るさの変化を吸収）
       - コントラスト変化への対応
       - 記述子の比較を公平に行える
       - SIFT では追加でクリッピング（0.2上限）も行う
    """
    print(answers)

# 解答を見るには以下のコメントを外して実行
# show_quiz_answers()

---

## ナビゲーション

- **前のノートブック**: [57. 三角測量と3D復元](57_triangulation_3d_reconstruction_v1.ipynb)
- **次のノートブック**: [59. SfM パイプライン基礎](59_sfm_pipeline_basics_v1.ipynb)
- **カリキュラム**: [CURRICULUM_UNIT_0.3.md](CURRICULUM_UNIT_0.3.md)